# 資料錯位修正與資料格式化

In [1]:
# 引入套件
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import os
import math
import time
import datetime
import json
# from datetime import datetime

In [2]:
# 設定讀取資料的路徑
path='D:\\桌面\\after_Clean_data\\'
# 將路徑中的檔案存成一個列表
file_list = os.listdir(path)
file_list.pop(-1) # 如果檔案列表中有奇怪的東西存在可以直接指定位置拿掉
file_list

['2020080500_after.json',
 '2020080501_after.json',
 '2020080502_after.json',
 '2020080503_after.json',
 '2020080504_after.json',
 '2020080505_after.json',
 '2020080506_after.json',
 '2020080507_after.json',
 '2020080508_after.json',
 '2020080509_after.json',
 '2020080510_after.json',
 '2020080511_after.json',
 '2020080512_after.json',
 '2020080513_after.json',
 '2020080514_after.json',
 '2020080515_after.json',
 '2020080516_after.json',
 '2020080517_after.json',
 '2020080518_after.json',
 '2020080519_after.json',
 '2020080520_after.json',
 '2020080521_after.json',
 '2020080522_after.json',
 '2020080523_after.json',
 '2020080600_after.json',
 '2020080601_after.json',
 '2020080602_after.json',
 '2020080603_after.json',
 '2020080604_after.json',
 '2020080605_after.json',
 '2020080606_after.json',
 '2020080607_after.json',
 '2020080608_after.json',
 '2020080609_after.json',
 '2020080610_after.json',
 '2020080611_after.json',
 '2020080612_after.json',
 '2020080613_after.json',
 '2020080614

In [3]:
# 這裡會將錯位的資料存回正確的檔案，但有一個條件是假如這個錯位的資料沒有對應的那個小時的檔案會直接報錯
# 輸入需要檔案路徑 日期 以及錯誤的資料dataframe
def comebine_Wrong_position_Data (path,Date,df_Yesterday):
    # 讀取錯位的原資料表
    df_Fix_error = pd.read_json(path+'資料合併_ver1\\'+Date+'_after.json')
    # 將錯誤的資料dataframe放在最後
    df_Fix_error = df_Fix_error.append(df_Yesterday,ignore_index=True)
    # 最後輸出成json
    df_Fix_error.to_json(path+'資料合併_ver1\\'+Date+'_after.json')

In [4]:
# 將input進來的dataframe找出錯位資料的位置並回傳錯位資料的index
def find_Index_wrong_position(correct_Time,error_Time,dataframe):
    # 將dataframe的大小存入變數
    def_shape = dataframe.shape
    # 將dataframe全部資料都跑一次，找出n與n+1位置的日期(到小時)是不是一樣，假如不一樣就回傳該筆資料的index
    for def_line in range(def_shape[0]-1):
        # 將每一筆資料的時間進行格式化
        tmp_time_Format_correct = pd.to_datetime(str(dataframe.iloc[def_line+1,0]))
        tmp_time_Format_error = pd.to_datetime(str(dataframe.iloc[def_line,0]))
        # 格式化成小時
        time_Format_c = tmp_time_Format_correct.strftime("%Y%m%d%H")
        time_Format_e = tmp_time_Format_error.strftime("%Y%m%d%H")
        # 比對n與n+1的時間是否不一樣
        if error_Time == time_Format_e and correct_Time == time_Format_c:
            return def_line

In [5]:
# 用for迴圈跑所有檔案列表中的檔案
for read_File_list in file_list:
#     get_This_filename = read_File_list.split('_')
    # 將檔名與路徑結合，變成該檔案的位置
    get_Path_name = path+read_File_list
    # 讀取該檔案的位置
    df_origin = pd.read_json(get_Path_name)
    # 將讀取進來的資料中的timestamp做unique取出不重複的值存成一個列表
    tmp_time = df_origin['Timestamp'].unique()
    # 定義一個list備用
    time_String=[]
    # 抓這個檔案裏面有沒有奇怪的時間，奇怪的就丟回去
    # for迴圈跑剛剛不重複的時間列表，並把每個時間格式化成小時，並找出不重複，
    # 假設只有一個值就代表沒有錯位的資料，如果有兩個值，就代表有錯位的資料。
    for time_Cut in tmp_time:
        # 將時間格式化
        tmp = pd.to_datetime(str(time_Cut))
        # 格式化到小時然後存入剛剛定義的list
        time_String.append(tmp.strftime("%Y%m%d%H"))
    # 取出不重複的值
    time_String = np.unique(time_String)
    # 檢查點
    print(read_File_list , time_String)
    
    # 定義兩個dataframe一個準備存正確的資料，另一個存錯位的資料
    df_This_correct = pd.DataFrame()
    df_YesterDay = pd.DataFrame()
    # 如果上面檢查時間的list裡的值只有一個，那代表沒有錯位的資料，直接輸出就好
    if len(time_String) == 1:
        print('No error position Data')
        # 輸出成json
        df_origin.to_json(path+'資料合併_ver1\\'+time_String[0]+'_after.json',orient ='records')
    # 如果檢查時間的list裡的值有兩個，那就代表有錯位的資料
    elif len(time_String) == 2:
        # 將dataframe帶入定義好的function找出錯位資料的index,並存在變數中
        error_Data_position = find_Index_wrong_position(time_String[1],time_String[0],df_origin)
        # 用存好的index和定義好的dataframe將正確的資料存進去
        df_This_correct = df_This_correct.append(df_origin[error_Data_position+1:])
        # 將錯誤的資料也存進dataframe
        df_YesterDay = df_YesterDay.append(df_origin[0:error_Data_position+1])
        print('There is an error position Data')
#         print(df_YesterDay)
#         print(df_This_correct)
        # 將正確資料輸出成json
        df_This_correct.to_json(path+'資料合併_ver1\\'+time_String[1]+'_after.json',orient ='records')
        # 將錯誤資料帶入另一個定義好的function
        try:
            comebine_Wrong_position_Data(path,time_String[0],df_YesterDay)
        except Exception as e:
            print('Error: ',e)
    else:
        print('有其他的Error')

2020080500_after.json ['2020080423' '2020080500']
There is an error position Data
Error:  Expected object or value
2020080501_after.json ['2020080500' '2020080501']
There is an error position Data
2020080502_after.json ['2020080501' '2020080502']
There is an error position Data
2020080503_after.json ['2020080502' '2020080503']
There is an error position Data
2020080504_after.json ['2020080503' '2020080504']
There is an error position Data
2020080505_after.json ['2020080504' '2020080505']
There is an error position Data
2020080506_after.json ['2020080505' '2020080506']
There is an error position Data
2020080507_after.json ['2020080506' '2020080507']
There is an error position Data
2020080508_after.json ['2020080507' '2020080508']
There is an error position Data
2020080509_after.json ['2020080508' '2020080509']
There is an error position Data
2020080510_after.json ['2020080509' '2020080510']
There is an error position Data
2020080511_after.json ['2020080510' '2020080511']
There is an err

There is an error position Data
2020080904_after.json ['2020080903' '2020080904']
There is an error position Data
2020080905_after.json ['2020080904' '2020080905']
There is an error position Data
2020080906_after.json ['2020080905' '2020080906']
There is an error position Data
2020080907_after.json ['2020080906' '2020080907']
There is an error position Data
2020080908_after.json ['2020080907' '2020080908']
There is an error position Data
2020080909_after.json ['2020080908' '2020080909']
There is an error position Data
2020080910_after.json ['2020080909' '2020080910']
There is an error position Data
2020080911_after.json ['2020080910' '2020080911']
There is an error position Data
2020080912_after.json ['2020080911' '2020080912']
There is an error position Data
2020080913_after.json ['2020080912' '2020080913']
There is an error position Data
2020080914_after.json ['2020080913' '2020080914']
There is an error position Data
2020080915_after.json ['2020080914' '2020080915']
There is an erro

# 第二次清理-自動化

In [7]:
# 定義一個function用來處理被pandas弄亂的資料格式
def clean_JsonData_twice(input_Path , file_Name , output_Path):
    # 讀取檔案
    df = pd.read_json(input_Path+file_Name)
    # 將timestamp存成一個list
    list_Timestamp = df['Timestamp'].tolist()
    # 將Z1存成一個list
    list_Z1 = df['Z1'].tolist()
    # 將Z2存成一個list
    list_Z2 = df['Z2'].tolist()
    # 讀取檔案並開啟寫入
    with open(output_Path+file_Name,'w') as rebu:
        # 計算timestamp的數量
        file_Len = len(list_Timestamp)
        # 先寫入前中括號和患行
        rebu.write('[\n')
        # 然後一筆一筆資料修改
        for i in range(file_Len):
            # 如果不是最後一行就要加逗點和換行
            if i < file_Len-1:
                # 將timestamp格式化並寫入
                rebu.writelines('{'+'"Timestamp"'+':"'+list_Timestamp[i].strftime("%Y-%m-%d %H:%M:%S")+'",'+
                                '"Z1"'+':'+str(list_Z1[i])+','+'"Z2"'+':'+str(list_Z2[i])+'},\n')
            # 如果該筆資料是最後一行，那就不加逗點只加換行
            elif i == file_Len-1:
                rebu.writelines('{'+'"Timestamp"'+':"'+list_Timestamp[i].strftime("%Y-%m-%d %H:%M:%S")+'",'+
                                '"Z1"'+':'+str(list_Z1[i])+','+'"Z2"'+':'+str(list_Z2[i])+'}\n')
        # 補上後中括號
        rebu.write(']')

In [8]:
input_Path = path+'資料合併_ver1\\'
output_Path = 'D:\\桌面\\clean_twice\\'
# 將file_list中的每一個檔案都跑一次
for i in file_list:
    # 呼叫定義的function把讀取路徑、檔名、輸出路徑放進去
    clean_JsonData_twice(input_Path,i,output_Path)